In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
import multiprocessing
from functools import partial
import timeit
#from scipy.spatial.distance import euclidean

In [2]:
## read gene exp data 
gene_exp = pd.read_csv("Gene_exp_norm.tsv",sep="\t", index_col="entity1" )
gene_exp.head()

,ENSG00000167578,ENSG00000078237,ENSG00000146083,ENSG00000158486,ENSG00000198242,ENSG00000134108,ENSG00000172137,ENSG00000167700,ENSG00000060642,ENSG00000182141,...,ENSG00000173930,ENSG00000107863,ENSG00000213782,ENSG00000146707,ENSG00000135094,ENSG00000009694,ENSG00000186115,ENSG00000105063,ENSG00000123685,ENSG00000181518
entity1,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01A,0.220424,0.091807,0.348096,0.000955,0.149053,0.075523,0.003335,0.109937,0.293197,0.384178,...,0.003264,0.109539,0.266748,0.279162,0.141285,0.000942,0.000000,0.399569,0.038784,0.0
TCGA-3C-AALI-01A,0.302636,0.071110,0.230751,0.199572,0.310995,0.265380,0.003954,0.097164,0.260287,0.209493,...,0.005310,0.045760,0.282749,0.084233,0.128204,0.001430,0.005622,0.585903,0.164510,0.0
TCGA-A2-A04N-01A,0.159254,0.213359,0.313224,0.027788,0.167568,0.281944,0.003263,0.206458,0.446931,0.242361,...,0.000485,0.159006,0.276830,0.202881,0.068045,0.008566,0.092370,0.212499,0.068655,0.0
TCGA-A2-A04P-01A,0.410988,0.075490,0.148639,0.014322,0.106746,0.178348,0.075258,0.088848,0.083922,0.439981,...,0.006245,0.112576,0.211774,0.064886,0.171326,0.002523,0.000000,0.196209,0.148864,0.0
TCGA-A2-A04Q-01A,0.282637,0.498424,0.213867,0.002229,0.166103,0.124329,0.044407,0.075296,0.119902,0.229810,...,0.006066,0.216021,0.311139,0.127619,0.188613,0.005419,0.000000,0.250735,0.226456,0.0


In [3]:
#gene_exp_CNN = gene_exp.drop(["Target_gene"],axis=1)

In [4]:
## import selected feature using statistical test
gene_exp_sel_feature = pd.read_csv("Gene_exp_2000_feautre.csv")
gene_exp_sel_feature.head()

,Gene Expression,Gene Expression (PLS-DA),DNA Methylation,DNA Methylation (PLS-DA),miRNA Expression,miRNA Expression (PLS-DA)
0,ENSG00000152684,ENSG00000152684,cg15761405,cg15761405,hsa-mir-625,hsa-mir-625
1,ENSG00000159450,ENSG00000179363,cg05022306,cg05022306,hsa-mir-4435-2,hsa-mir-4435-2
2,ENSG00000179363,ENSG00000102144,cg16276070,cg16276070,hsa-mir-144,hsa-mir-144
3,ENSG00000104131,ENSG00000163900,cg25177139,cg25177139,hsa-mir-31,hsa-mir-31
4,ENSG00000105270,ENSG00000165125,cg10038087,cg13594711,hsa-mir-4658,hsa-mir-4658


In [5]:
# create list of selscted feature
gene_exp_sel_feature_list = gene_exp_sel_feature[["Gene Expression (PLS-DA)"]].squeeze().to_list()

In [6]:
len(gene_exp_sel_feature_list)

2000

In [7]:
gene_exp_fil = gene_exp[gene_exp_sel_feature_list]

In [8]:
gene_exp_fil.shape

(302, 2000)

In [9]:
## final gene expreesion data with 302 rows and 2000 selected freature.
gene_exp_fil.head()

,ENSG00000152684,ENSG00000179363,ENSG00000102144,ENSG00000163900,ENSG00000165125,ENSG00000131015,ENSG00000080824,ENSG00000077585,ENSG00000232119,ENSG00000082458,...,ENSG00000197705,ENSG00000122852,ENSG00000105251,ENSG00000071991,ENSG00000198721,ENSG00000269466,ENSG00000121743,ENSG00000103154,ENSG00000157168,ENSG00000175564
entity1,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01A,0.375683,0.024997,0.113926,0.214114,0.076280,0.023396,0.064816,0.043168,0.182239,0.666743,...,0.012969,0.0,0.016920,0.000000,0.072417,0.000000,0.032073,0.004168,0.000512,0.040141
TCGA-3C-AALI-01A,0.184641,0.063264,0.154970,0.128633,0.110873,0.380025,0.100665,0.092063,0.064039,0.452983,...,0.001688,0.0,0.020554,0.010721,0.074641,0.004102,0.004870,0.009820,0.016988,0.159668
TCGA-A2-A04N-01A,0.178011,0.043310,0.086744,0.169166,0.014797,0.029821,0.108310,0.152118,0.173433,0.212901,...,0.015023,0.0,0.000000,0.000075,0.162090,0.000000,0.047790,0.005205,0.015004,0.056250
TCGA-A2-A04P-01A,0.151565,1.000000,0.015756,0.114135,0.192546,0.371124,0.070956,0.035062,0.290061,0.000000,...,0.009926,0.0,0.010878,0.001467,0.000919,0.000000,0.021767,0.016038,0.001281,0.085472
TCGA-A2-A04Q-01A,0.302271,0.016678,0.066349,0.225975,0.080364,0.032125,0.085960,0.068935,0.210800,0.078516,...,0.021509,0.0,0.030480,0.000144,0.092213,0.000000,0.006420,0.001958,0.010085,0.048018


In [11]:
## create matrix with 2000 feature to append distance between eveery feature.
corr_df = pd.DataFrame(0, index=gene_exp_fil.columns, columns=gene_exp_fil.columns)

In [12]:
corr_df.head()

,ENSG00000152684,ENSG00000179363,ENSG00000102144,ENSG00000163900,ENSG00000165125,ENSG00000131015,ENSG00000080824,ENSG00000077585,ENSG00000232119,ENSG00000082458,...,ENSG00000197705,ENSG00000122852,ENSG00000105251,ENSG00000071991,ENSG00000198721,ENSG00000269466,ENSG00000121743,ENSG00000103154,ENSG00000157168,ENSG00000175564
ENSG00000152684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000179363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000102144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000163900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000165125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
corr_df_dup = corr_df.copy()

In [14]:
corr_df.shape

(2000, 2000)

In [19]:
%pwd

'/home/seq/kunalt/CNN/CNN/op2'

In [16]:
%cd op2

/home/seq/kunalt/CNN/CNN/op2


## data broadcast

In [97]:
np_gene_exp = gene_exp_fil.to_numpy()
np_gene_exp

array([[3.75682550e-01, 2.49971284e-02, 1.13926399e-01, ...,
        4.16758521e-03, 5.12406801e-04, 4.01409837e-02],
       [1.84640575e-01, 6.32641166e-02, 1.54970276e-01, ...,
        9.81963615e-03, 1.69884403e-02, 1.59668212e-01],
       [1.78010672e-01, 4.33099066e-02, 8.67444022e-02, ...,
        5.20475147e-03, 1.50037119e-02, 5.62503550e-02],
       ...,
       [7.05643041e-01, 2.73288491e-01, 2.97899979e-01, ...,
        2.26295402e-03, 1.55612187e-04, 7.52119754e-02],
       [2.78114401e-01, 2.46556232e-01, 4.77081784e-02, ...,
        1.33784029e-02, 8.78444481e-03, 7.59220273e-02],
       [3.80024195e-01, 1.03043555e-01, 7.76156793e-02, ...,
        2.30838274e-03, 1.07592804e-01, 5.67128555e-02]])

In [98]:
np_gene_exp.shape

(302, 2000)

In [99]:
ex = np_gene_exp[0]*np.ones((2000,2000))

In [100]:
ex

array([[0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098],
       [0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098],
       [0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098],
       ...,
       [0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098],
       [0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098],
       [0.37568255, 0.02499713, 0.1139264 , ..., 0.00416759, 0.00051241,
        0.04014098]])

In [39]:
ex.T

array([[0.37568255, 0.37568255, 0.37568255, ..., 0.37568255, 0.37568255,
        0.37568255],
       [0.02499713, 0.02499713, 0.02499713, ..., 0.02499713, 0.02499713,
        0.02499713],
       [0.1139264 , 0.1139264 , 0.1139264 , ..., 0.1139264 , 0.1139264 ,
        0.1139264 ],
       ...,
       [0.00416759, 0.00416759, 0.00416759, ..., 0.00416759, 0.00416759,
        0.00416759],
       [0.00051241, 0.00051241, 0.00051241, ..., 0.00051241, 0.00051241,
        0.00051241],
       [0.04014098, 0.04014098, 0.04014098, ..., 0.04014098, 0.04014098,
        0.04014098]])

In [24]:
ex.shape

(2000, 2000)

In [40]:
np.linalg.norm(ex- ex.T)

561.6685922590544

In [101]:
aww = np.sqrt(np.power(ex-ex.T,2))

In [102]:
aww

array([[0.        , 0.35068542, 0.26175615, ..., 0.37151496, 0.37517014,
        0.33554157],
       [0.35068542, 0.        , 0.08892927, ..., 0.02082954, 0.02448472,
        0.01514386],
       [0.26175615, 0.08892927, 0.        , ..., 0.10975881, 0.11341399,
        0.07378542],
       ...,
       [0.37151496, 0.02082954, 0.10975881, ..., 0.        , 0.00365518,
        0.0359734 ],
       [0.37517014, 0.02448472, 0.11341399, ..., 0.00365518, 0.        ,
        0.03962858],
       [0.33554157, 0.01514386, 0.07378542, ..., 0.0359734 , 0.03962858,
        0.        ]])

In [48]:
aww1= np.random.rand(100,100)

# with Parallel processing

In [20]:
def calculate_ranks(chunk,unique_values, sorted_indices):
    ranks = np.empty_like(chunk)
    #unique_values, value_counts = np.unique(chunk, return_counts=True)
    #sorted_indices = np.argsort(unique_values)
    for k, value in enumerate(unique_values[sorted_indices]):
        #print(chunk)
        indices = np.where(chunk == value)
        ranks[indices] = k
    return ranks

In [ ]:
#start = timeit.default_timer()
for index, row in gene_exp_fil.iterrows():
    start = timeit.default_timer()
    corr_df_dup = corr_df.copy()
    for index1, i in enumerate(gene_exp_sel_feature_list):
        for index2, j in enumerate(gene_exp_sel_feature_list):
            if index1<index2:
                pass
            elif i==j:
                corr_df_dup.loc[i,j]=0
            else:
                corr_df_dup.loc[i,j] = np.linalg.norm(row[i]- row[j])
                corr_df_dup.loc[j,i]=corr_df_dup.loc[i,j]
                #print ('1')###################
                # flatten the matrix into a 1D array
    flat_matrix = corr_df_dup.to_numpy().flatten()
    unique_values, value_counts = np.unique(flat_matrix, return_counts=True)
    # get the sorted indices of the unique values
    sorted_indices = np.argsort(unique_values)
    func = partial(calculate_ranks, unique_values=unique_values, sorted_indices=sorted_indices)

    pool = multiprocessing.Pool()
    results = pool.map(func, np.array_split(flat_matrix, multiprocessing.cpu_count()))

    ranks = np.concatenate(results).reshape(corr_df_dup.shape)
    pd.DataFrame(ranks, columns=corr_df_dup.columns, index=corr_df_dup.index).to_csv(f'tsv/{index}.tsv',sep="\t")
    # plot heatmap and saving using index name
    
    plt.imshow( ranks , cmap = 'Greys' , interpolation = 'nearest' )
    ## save image
    plt.savefig(f'{index}')
    stop = timeit.default_timer()
    print(stop - start)
    print("Image for "+ index + " genrated")
#stop = timeit.default_timer()

383.32904326636344
Image for TCGA-E2-A150-01A genrated
380.0256337733008
Image for TCGA-E2-A152-01A genrated
381.87886520894244
Image for TCGA-E2-A15M-01A genrated
379.6334719359875
Image for TCGA-E2-A1AZ-01A genrated
387.2665531639941
Image for TCGA-E2-A1B1-01A genrated
380.34334514895454
Image for TCGA-E2-A1B4-01A genrated
383.0205215238966
Image for TCGA-E2-A1IE-01A genrated
384.2499491730705
Image for TCGA-E2-A1IG-01A genrated
388.0828187619336
Image for TCGA-E2-A1IO-01A genrated
384.3876619138755
Image for TCGA-E2-A1L7-01A genrated
384.28540439205244
Image for TCGA-E2-A1L8-01A genrated
380.13208072725683
Image for TCGA-E2-A1LB-01A genrated


In [18]:
np_gene_exp = gene_exp_fil.to_numpy()

In [ ]:
for row in np_gene_exp:
    start = timeit.default_timer()
    ex = row*np.ones((2000,2000))
    corr_df_dup = np.sqrt(np.power(ex-ex.T,2))
                #print ('1')###################
    # flatten the matrix into a 1D array
    flat_matrix = corr_df_dup.flatten()
    unique_values, value_counts = np.unique(flat_matrix, return_counts=True)
    # get the sorted indices of the unique values
    sorted_indices = np.argsort(unique_values)
    func = partial(calculate_ranks, unique_values=unique_values, sorted_indices=sorted_indices)

    pool = multiprocessing.Pool()
    results = pool.map(func, np.array_split(flat_matrix, multiprocessing.cpu_count()))

    ranks = np.concatenate(results).reshape(corr_df_dup.shape)
    pd.DataFrame(ranks, columns=corr_df_dup.columns, index=corr_df_dup.index).to_csv(f'tsv/{index}_2.tsv',sep="\t")
    # plot heatmap and saving using index name
    
    plt.imshow( ranks , cmap = 'Greys' , interpolation = 'nearest' )
    plt.savefig(f'{index}_2')
    stop = timeit.default_timer()
    print(stop - start)
    print("Image for "+ index + " genrated")